In [3]:
import tensorflow as tf
import os
from tqdm import tqdm
import urllib.request as req

In [2]:
!pip install tqdm

You should consider upgrading via the 'C:\Users\pallavi.saxena\Documents\Mine\iNeuron\FullStackDataScienceNovember2021\FastTrack\3.DataScienceMasters\2.DL\1.FLive\Lecture16_NewCNNclassification\Codebase\CNN-classifier-FSDS\env\python.exe -m pip install --upgrade pip' command.



     ---------------------------------------- 76.6/76.6 KB 1.4 MB/s eta 0:00:00
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)


In [4]:
SOURCE_URL = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip"

In [5]:
DESTINATION = "data"
os.makedirs(DESTINATION, exist_ok=True)
data_file = "data.zip"

DESTINATION_ZIP_PATH = os.path.join(DESTINATION, data_file)

In [6]:
filename, headers = req.urlretrieve(SOURCE_URL, DESTINATION_ZIP_PATH)

In [7]:
filename

'data\\data.zip'

In [8]:
print(headers)

Content-Type: application/octet-stream
Accept-Ranges: bytes
Server: Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0
x-ms-blob-content-md5: l2WJyK3tt5/f2azyQ9eE8w==
Last-Modified: Sat, 13 Oct 2018 12:14:22 GMT
ETag: "0x8D6310568FD1DF6"
Content-Length: 824894548
Date: Sun, 03 Apr 2022 02:39:53 GMT
Connection: close




In [9]:
os.getcwd()

'C:\\Users\\pallavi.saxena\\Documents\\Mine\\iNeuron\\FullStackDataScienceNovember2021\\FastTrack\\3.DataScienceMasters\\2.DL\\1.FLive\\Lecture16_NewCNNclassification\\Codebase\\CNN-classifier-FSDS\\notebook'